<a href="https://colab.research.google.com/github/vrangayyan6/Machine-Learning/blob/main/fastai2_ulmfit_multiclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip show fastai

Name: fastai
Version: 2.0.16
Summary: fastai simplifies training fast and accurate neural nets using modern best practices
Home-page: https://github.com/fastai/fastai/tree/master/
Author: Jeremy Howard, Sylvain Gugger, and contributors
Author-email: info@fast.ai
License: Apache Software License 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: spacy, pyyaml, packaging, matplotlib, torch, pillow, fastcore, fastprogress, requests, pip, scikit-learn, scipy, torchvision, pandas
Required-by: 


In [ ]:
!pip install fastai --upgrade

Requirement already up-to-date: fastai in /usr/local/lib/python3.6/dist-packages (2.0.16)


In [ ]:
from fastai.text.all import *

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Game reviews dataset
https://drive.google.com/drive/u/0/folders/1HwJOw2kPpeFzpSvJL_qMmINrtL5lq6RH filename: dataset_version2_commonsensemedia_reviews.csv

In [ ]:
combine_path = "/content/drive/My Drive/Omdena/dataset_version2_commonsensemedia_reviews.csv"
input_df = pd.read_csv(combine_path, header=0)
print(input_df.shape)
input_df.head()

(40433, 17)


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,game_name,review,user_name,reviewer_type,type,Positive Messages,Positive Role Models & Representations,Ease of Play,Violence,Sex,Language,Consumerism,"Drinking, Drugs & Smoking",review_sentiment,risk_level,risk_level_encoded,type_encoded
0,Flickr,Depends what photos you are looking at. If your looking at just having a place to store photo's then choose this place.,ICEYWARM,Adult,website,3.0,3.0,3.0,4.0,3.0,3.0,3.0,0.0,Flicker,high_risk,1,1
1,Flickr,"I hate it, its amazing what people say and post these days. If you have kids or your a kid, remember once you send something over the internet, who knows who will get it.",2442hayhay2442,"Parent of a 8, 11, and 13-year-old",website,3.0,3.0,3.0,4.0,3.0,3.0,3.0,0.0,age???,high_risk,1,1
2,Flickr,great site for all ages.,Kejr,Adult,website,3.0,3.0,3.0,4.0,3.0,3.0,3.0,0.0,NaN,high_risk,1,1
3,Flickr,"Flickr can be a rough place, but I choose to stay away from most of that and focus on people's photos of nature and such.\nI find non-photographers on Flickr rather aggravating, as they may post some obscene photo in a nature group, etc.\nMost often, it's teens that get the photography bug, not children. I recommend using this website once you've gotten yourself hooked on the photography hobby.",MisterObvious,"Teen, 14 years old",website,3.0,3.0,3.0,4.0,3.0,3.0,3.0,0.0,For Photographers Only,high_risk,1,1
4,Flickr,"Flickr is a good photo sharing site, there's not much inappropriate content, it's not inappropriate like Deviantart and it automatically blocks inappropriate content I believe.",TheBombFunn,"Teen, 13 years old",website,3.0,3.0,3.0,4.0,3.0,3.0,3.0,0.0,Great photo sharing website,high_risk,1,1


In [ ]:
print(input_df.columns)

Index(['game_name', 'review', 'user_name', 'reviewer_type', 'type',
       'Positive Messages', 'Positive Role Models & Representations',
       'Ease of Play', 'Violence', 'Sex', 'Language', 'Consumerism',
       'Drinking, Drugs & Smoking', 'review_sentiment', 'risk_level',
       'risk_level_encoded', 'type_encoded'],
      dtype='object')


In [ ]:
# filter reviews of Adults input_df[input_df.reviewer_type == 'Adult']
combine_df = input_df.loc[:,['review', 'Sex']].rename(columns={"review": "Comments", "Sex": "label"})
print(combine_df.isnull().sum())

Comments      5
label       238
dtype: int64


In [ ]:
combine_df = combine_df.dropna().reset_index(drop=True)
print(combine_df.isnull().sum())
print(combine_df['label'].value_counts())

Comments    0
label       0
dtype: int64
0.0    18391
3.0    10675
1.0     3904
4.0     3244
2.0     2464
5.0     1515
Name: label, dtype: int64


In [ ]:
combine_df = combine_df.groupby('label').apply(lambda x: x.sample(n=1500, replace=True, random_state=10))
print(combine_df.shape)
print(combine_df['label'].value_counts())
# max comment length
print('max ', max(combine_df['Comments'].apply(len)))

combine_df.head()

(9000, 2)
5.0    1500
4.0    1500
3.0    1500
2.0    1500
1.0    1500
0.0    1500
Name: label, dtype: int64
max  9499


Comments  label
label                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
0.0   39261                                                                                                                                                                                                                                                                                                                                                                                                                            HOLY CRAP THIS GAME SCARED THE CRAP OUT OF ME!!!very violent and could give a child night terrors please don't let a kid who gets scared easily play this it could mess them up pshycologicly    0.0
      19002                                                                                                               If you like scary games like Left 4 Dead or Slenderman. I recommend giving Five Night's at Freddy's a try. It has simple mechanics and there jumpscares are not even scary in my opinion, there just bobbing heads. But it's backstory is very dark. Also for parents, there is this dude named golden Freddy. You can get by his jumpscare by selecting on custom night: 1987. You will see his black deep eyes following with a lower pitch version of the jumpscare sound, he also makes your game ded.    0.0
      39316                                                                                                                                                                                                                                                                                                                              Play is very good and follows movie well. Graphics and sound are excellent in MOST scenes. Could have a little more ability to go your own way. However there is more than enough possibilities to entertain. Only wish it was longer. End comes too quickly. All in all an excellent game.    0.0
      19000  I am an avid fan of this game and have been in the lore since the first game. Something I need to address is all of the parents who are hating on the game just because they dont like it or they only watched the theories and the backstories.\nIs the game suspenseful? Yes, 100%. Does the game itself show any violence or gore? No. Only the theories and the backstory to why the events happen in the game are disturbing, but extremely intriguing. The only blood that is shown in the game franchise is at the very end of the third game when you see a purple man go inside a spring-lock suit, and pro...    0.0
      22263                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 Very fun    0.0

#Using high level text classifier API

In [ ]:
dls = TextDataLoaders.from_df(combine_df, valid_pct=0.1)

In [ ]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [ ]:
learn.fine_tune(4, 1e-02)

epoch,train_loss,valid_loss,accuracy,time
0,1.843424,1.679998,0.315556,00:13


epoch,train_loss,valid_loss,accuracy,time
0,1.626588,1.628839,0.365556,00:28
1,1.473397,1.403804,0.463333,00:28
2,1.261811,1.286699,0.516667,00:28
3,1.068821,1.295907,0.510000,00:28


In [ ]:
learn.fine_tune(4, 1e-02)

epoch,train_loss,valid_loss,accuracy,time
0,1.096192,1.416026,0.484444,00:14


epoch,train_loss,valid_loss,accuracy,time
0,1.093140,1.332903,0.496667,00:28
1,1.003061,1.375171,0.518889,00:28
2,0.843615,1.268450,0.557778,00:28
3,0.642742,1.260896,0.550000,00:28


In [ ]:
preds, targets = learn.get_preds()
predictions = np.argmax(preds, axis=1)
#pd.crosstab(predictions, targets, rownames=['predictions'], colnames=['targets'], margins=True)

from sklearn.metrics import classification_report
print(classification_report(targets, predictions))

              precision    recall  f1-score   support

           0       0.30      0.36      0.33       155
           1       0.65      0.51      0.57       163
           2       0.59      0.51      0.55       150
           3       0.54      0.53      0.53       154
           4       0.54      0.63      0.59       126
           5       0.73      0.78      0.76       152

    accuracy                           0.55       900
   macro avg       0.56      0.55      0.55       900
weighted avg       0.56      0.55      0.55       900



# Predict on our YouTube dataset

In [ ]:
predict_path = "/content/drive/My Drive/Omdena/YoutubeComment.csv"
predict_data_df = pd.read_csv(predict_path)
predict_data_df.head()

,User,Comments
0,Världens Bästa Bil,Lol just close the screen
1,pamela bibby,why are bullies so creul why do so many people bully and why cant people leave you alone
2,Romina Martinez,"It&#39;s difficult to hear these bad experiences, I have to make a story for the University about it. I&#39;d also been bullied when I was in primary school and then when I was in my first year of secondary school. it was really bad because I felt alone, nobody wanted to talk with me. they said bad things of me, I wanted to change school but I couldn&#39;t so I had to be with them until I had finished my first year. Then, everything had changed. I had made lots of friends and I get really good grades and now I&#39;m in my second year of university, it was difficult when my classmates told ..."
3,Ogarite Kattan,"The hardest part when you&#39;re bullied, is that you have to hurt and bleed alone sometimes... You start to believe what others say about you , you&#39;re lost and you just don&#39;t know what to do in this cruel world ..."
4,Luke Allan,Bullying is awful. I myself has been experiencing this. It has caused me to not respect myself and feel like I will never be good at anything. Even as an adult I feel scared and feel a bit scared to try and do better for myself. However what I also learned to do to help is to surround youreself with those who love you and care for you.


In [ ]:
pl = predict_data_df.Comments.to_list()
for p in pl[:10]:
  print(p, "\n", learn.predict(p), "\n")

Lol just close the screen 
 ('0.0', tensor(0), tensor([0.5170, 0.2463, 0.0832, 0.1328, 0.0160, 0.0047])) 



why are bullies so creul why do so many people bully and why cant people leave you alone 
 ('1.0', tensor(1), tensor([0.0966, 0.5383, 0.1723, 0.1873, 0.0031, 0.0023])) 



It&#39;s difficult to hear these bad experiences, I have to make a story for the University about it. I&#39;d also been bullied when I was in primary school and then when I was in my first year of secondary school. it was really bad because I felt alone, nobody wanted to talk with me. they said bad things of me, I wanted to change school but I couldn&#39;t so I had to be with them until I had finished my first year. Then, everything had changed. I had made lots of friends and I get really good grades and now I&#39;m in my second year of university, it was difficult when my classmates told me I would never have a future but today I can tell that I gave a good lesson to all of them. 
 ('1.0', tensor(1), tensor([0.2214, 0.3563, 0.2677, 0.1101, 0.0360, 0.0085])) 



The hardest part when you&#39;re bullied, is that you have to hurt and bleed alone sometimes...  You start to believe what others say about you , you&#39;re lost and you just don&#39;t know what to do in this cruel world ... 
 ('0.0', tensor(0), tensor([0.5234, 0.2060, 0.1322, 0.1002, 0.0278, 0.0104])) 



Bullying is awful. I myself has been experiencing this. It has caused me to not respect myself and feel like I will never be good at anything. Even as an adult I feel scared and feel a bit scared to try and do better for myself. However what I also learned to do to help is to surround youreself with those who love you and care for you. 
 ('1.0', tensor(1), tensor([0.0777, 0.7556, 0.0328, 0.0687, 0.0405, 0.0247])) 



I get cyber bullied repeatedly. It ends, then starts again. You&#39;re waiting for who comes next, Its like<br />fighting a wave of never ending enemies. <a href="http://www.youtube.com/results?search_query=%23bullyingneedstostopnow">#bullyingneedstostopnow</a> 
 ('1.0', tensor(1), tensor([0.1816, 0.3354, 0.1214, 0.2305, 0.1047, 0.0264])) 



I&#39;m trying to spread the idea to stand up and say no! 
 ('0.0', tensor(0), tensor([0.5080, 0.1369, 0.1468, 0.1447, 0.0329, 0.0307])) 



I cry every night and smile every day 😃😃😃😃 
 ('0.0', tensor(0), tensor([0.6187, 0.2109, 0.0641, 0.0396, 0.0587, 0.0081])) 



I&#39;m bullying to every day...and I must live witch that 😔 so what I&#39;m gonna do?? NOTHING 😢 I know what that mean but I try to smile 😃😃❤ 
 ('1.0', tensor(1), tensor([0.1538, 0.5015, 0.1128, 0.2138, 0.0091, 0.0090])) 



Lol just close the screen 
 ('0.0', tensor(0), tensor([0.5170, 0.2463, 0.0832, 0.1328, 0.0160, 0.0047])) 

